<a href="https://colab.research.google.com/github/gagan3012/llama-tdlr/blob/master/LLMTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructional Finetuning

In [ ]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

In [ ]:
from trl import SFTTrainer
from datasets import load_dataset
from random import randrange
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, AutoTokenizer

dataset = load_dataset("databricks/databricks-dolly-15k")

### Alpaca Format

In [ ]:
def format_instruction(sample):
    if sample['context'] == '':
        sample['text'] = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
        ### Instruction:
        {sample['instruction']}

        ### Response:
        {sample['response']}
        """
    else:
        sample['text'] = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
        ### Instruction:
        {sample['instruction']}

        ### Input:
        {sample['context']}

        ### Response:
        {sample['response']}
        """
    return sample

In [ ]:
dataset = dataset.map(format_instruction)
print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])

### Model Loading

In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### Setup

In [ ]:
args = TrainingArguments(
    output_dir="llama-2-dolly",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    save_total_limit=2,
    save_steps=1000,
)

### Train

In [ ]:
max_seq_length = 2048  # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    args=args,
    dataset_text_field="text",
)


In [ ]:
trainer.train()

# Lora PEFT

In [ ]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

In [ ]:
from trl import SFTTrainer
from datasets import load_dataset
from random import randrange
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, AutoTokenizer

dataset = load_dataset("databricks/databricks-dolly-15k")

### Alpaca Format

In [ ]:
def format_instruction(sample):
    if sample['context'] == '':
        sample['text'] = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
        ### Instruction:
        {sample['instruction']}

        ### Response:
        {sample['response']}
        """
    else:
        sample['text'] = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
        ### Instruction:
        {sample['instruction']}

        ### Input:
        {sample['context']}

        ### Response:
        {sample['response']}
        """
    return sample

In [ ]:
dataset = dataset.map(format_instruction)
print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])

### Model Loading

In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)

### Setup

In [ ]:
args = TrainingArguments(
    output_dir="llama-2-dolly",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    save_total_limit=2,
    save_steps=1000,
)

### Train

In [ ]:
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(
            args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        kwargs["model"].save_pretrained(checkpoint_folder)

        pytorch_model_path = os.path.join(
            checkpoint_folder, "pytorch_model.bin")
        torch.save({}, pytorch_model_path)
        return control


class LoadBestPeftModelCallback(TrainerCallback):
    def on_train_end(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        print(
            f"Loading best peft model from {state.best_model_checkpoint} (score: {state.best_metric}).")
        best_model_path = os.path.join(
            state.best_model_checkpoint, "adapter_model.bin")
        adapters_weights = torch.load(best_model_path)
        model = kwargs["model"]
        set_peft_model_state_dict(model, adapters_weights)
        return control

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length=max_seq_length,
    peft_config=peft_config,
    tokenizer=tokenizer,
    packing=True,
    args=args,
    dataset_text_field="text",
    callbacks=[LoadBestPeftModelCallback(), SavePeftModelCallback()],
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

### Merge and Save Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)

model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
)

# Merge LoRA and base model
merged_model = model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained(args.output_dir, safe_serialization=True)
tokenizer.save_pretrained(args.output_dir)

# QLora PEFT

In [ ]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

In [ ]:
from trl import SFTTrainer
from datasets import load_dataset
from random import randrange
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, AutoTokenizer

dataset = load_dataset("databricks/databricks-dolly-15k")

### Alpaca Format

In [ ]:
def format_instruction(sample):
    if sample['context'] == '':
        sample['text'] = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
        ### Instruction:
        {sample['instruction']}

        ### Response:
        {sample['response']}
        """
    else:
        sample['text'] = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
        ### Instruction:
        {sample['instruction']}

        ### Input:
        {sample['context']}

        ### Response:
        {sample['response']}
        """
    return sample

In [ ]:
dataset = dataset.map(format_instruction)
print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])

### Model Loading

In [ ]:
# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             use_cache=False,
                                             device_map="auto")
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

### Setup

In [ ]:
args = TrainingArguments(
    output_dir="llama-2-dolly",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    save_total_limit=2,
    save_steps=1000,
)

### Train

In [ ]:
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(
            args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        kwargs["model"].save_pretrained(checkpoint_folder)

        pytorch_model_path = os.path.join(
            checkpoint_folder, "pytorch_model.bin")
        torch.save({}, pytorch_model_path)
        return control


class LoadBestPeftModelCallback(TrainerCallback):
    def on_train_end(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        print(
            f"Loading best peft model from {state.best_model_checkpoint} (score: {state.best_metric}).")
        best_model_path = os.path.join(
            state.best_model_checkpoint, "adapter_model.bin")
        adapters_weights = torch.load(best_model_path)
        model = kwargs["model"]
        set_peft_model_state_dict(model, adapters_weights)
        return control

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length=max_seq_length,
    peft_config=peft_config,
    tokenizer=tokenizer,
    packing=True,
    args=args,
    dataset_text_field="text",
    callbacks=[LoadBestPeftModelCallback(), SavePeftModelCallback()],
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

### Merge and Save Model

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)

# Merge LoRA and base model
merged_model = model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained(args.output_dir, safe_serialization=True)
tokenizer.save_pretrained(args.output_dir)

# Inference

In [ ]:
!pip install vllm

In [ ]:
from vllm import LLM, SamplingParams
# Sample prompts.
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=100)

# Create an LLM.
llm = LLM(model=args.output_dir)
# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")